In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV
from scipy.stats import randint
from sklearn import preprocessing
import category_encoders as ce
from sklearn.model_selection import train_test_split

complete_dataset_path = '/Users/carloscarvalho/Downloads/microdados_enem_2021 2/DADOS/MICRODADOS_ENEM_2021.csv'
fraction_dataset_path = '/Users/carloscarvalho/Downloads/splitcsv-476aa139-fb59-4486-8b15-e84e386d7d24-results/MICRODADOS_ENEM_2021-1.csv'

df = pd.read_csv(complete_dataset_path, encoding="ISO-8859-1")


In [2]:
# Target = Math Exam grade
# Features = Info that is related to brazil's inequality
# and not related to a student's effort

In [3]:
df[df.columns[0].split(';')] = df[df.columns[0]].str.split(';', expand=True)
df = df.drop(df.columns[0], axis=1)

df['target'] = df['NU_NOTA_MT']
features = ['TP_SEXO', 'TP_COR_RACA', 'TP_ESCOLA', 'TP_LOCALIZACAO_ESC', 'Q001', 'Q002', 'Q003', 'Q004', 'Q005', 'Q006', 'Q007', 'Q008', 'Q009', 'Q010', 'Q011', 'Q012', 'Q013', 'Q014', 'Q015', 'Q016', 'Q017', 'Q018', 'Q019', 'Q020', 'Q021', 'Q022', 'Q023', 'Q024', 'Q025']

df = df[features + ['target']]
df = df.drop(df[df['target'] == ''].index)
df['target'] = df['target'].astype(float)

print(df.count())

TP_SEXO               2245844
TP_COR_RACA           2245844
TP_ESCOLA             2245844
TP_LOCALIZACAO_ESC    2245844
Q001                  2245844
Q002                  2245844
Q003                  2245844
Q004                  2245844
Q005                  2245844
Q006                  2245844
Q007                  2245844
Q008                  2245844
Q009                  2245844
Q010                  2245844
Q011                  2245844
Q012                  2245844
Q013                  2245844
Q014                  2245844
Q015                  2245844
Q016                  2245844
Q017                  2245844
Q018                  2245844
Q019                  2245844
Q020                  2245844
Q021                  2245844
Q022                  2245844
Q023                  2245844
Q024                  2245844
Q025                  2245844
target                2245844
dtype: int64


In [4]:
# split the dataset into predictor variables (X) and target variable (y)

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

X_train = train_df.drop('target', axis=1)
X_test = test_df.drop('target', axis=1)
y_train = train_df['target']
y_test = test_df['target']
    
target_encoder = ce.TargetEncoder(cols=X_train.columns)
target_encoder.fit(X_train, y_train)

X_train = target_encoder.transform(X_train)
X_test = target_encoder.transform(X_test)

model = RandomForestRegressor(max_depth=9).fit(X_train, y_train)

# calculate the R-squared
r_squared = r2_score(y_train, model.predict(X_train))
print("Train R-squared: ", r_squared)

r_squared = r2_score(y_test, model.predict(X_test))
print("Test R-squared: ", r_squared)



model = RandomForestRegressor(max_depth=10).fit(X_train, y_train)

# calculate the R-squared
r_squared = r2_score(y_train, model.predict(X_train))
print("Train R-squared: ", r_squared)

r_squared = r2_score(y_test, model.predict(X_test))
print("Test R-squared: ", r_squared)

model = RandomForestRegressor(max_depth=11).fit(X_train, y_train)

# calculate the R-squared
r_squared = r2_score(y_train, model.predict(X_train))
print("Train R-squared: ", r_squared)

r_squared = r2_score(y_test, model.predict(X_test))
print("Test R-squared: ", r_squared)




model = xgb.XGBRegressor().fit(X_train, y_train)

# calculate the R-squared
r_squared = r2_score(y_train, model.predict(X_train))
print("Train R-squared: ", r_squared)

r_squared = r2_score(y_test, model.predict(X_test))
print("Test R-squared: ", r_squared)

Train R-squared:  0.27784027355847085
Test R-squared:  0.2773029286390841
Train R-squared:  0.2830822686199085
Test R-squared:  0.28025084749287676
Train R-squared:  0.28959853837252547
Test R-squared:  0.28271508709991344
Train R-squared:  0.3013629726165362
Test R-squared:  0.2956894049761277


In [ ]:
params_grid = {
    'learning_rate': [0.05, 0.1, 0.15],
    'max_depth': [3, 4, 5],
    'min_child_weight': [1, 3, 5],
    'gamma': [0, 0.1, 0.2],
    'subsample': [0.6, 0.7, 0.8, 0.9],
    'colsample_bytree': [0.6, 0.7, 0.8, 0.9],
    'reg_alpha': [0, 0.1, 0.5, 1],
    'reg_lambda': [0.01, 0.1, 1, 10]
}

xgb_model = xgb.XGBRegressor()

# Initialize GridSearchCV for hyperparameter tuning
grid_search = GridSearchCV(estimator=xgb_model, param_grid=params_grid, cv=5, n_jobs=-1, scoring='neg_mean_squared_error')

# Fit GridSearchCV on the train set
grid_search.fit(X_train, y_train)

# Print the best parameters and score found during GridSearchCV
print("Best parameters:", grid_search.best_params_)
print("Best score:", -grid_search.best_score_)

# Train the XGBoost model with the best parameters found
best_xgb_model = xgb.XGBRegressor(**grid_search.best_params_)
best_xgb_model.fit(X_train, y_train)

r_squared = r2_score(y_train, best_xgb_model.predict(X_train))
print("Train R-squared: ", r_squared)

r_squared = r2_score(y_test, best_xgb_model.predict(X_test))
print("Test R-squared: ", r_squared)


In [ ]:
train_df[X_train.columns] = target_encoder.transform(train_df[X_train.columns])

print(train_df.corr())